In [5]:
from langgraph.graph import StateGraph, START, END
from langgraph.store.base import IndexConfig
from langchain.embeddings import init_embeddings
import os
from xmlrpc import client
from azure.cosmos import CosmosClient, PartitionKey
from cosmos_store import CosmosDBStore
from azure.identity import ClientSecretCredential
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential
import uuid
from langgraph.graph import MessagesState
from cosmos_store import CosmosDBStore
from langchain_openai import AzureChatOpenAI



keyVaultName = "akvlab00"
KVUri = f"https://{keyVaultName}.vault.azure.net"

credential = DefaultAzureCredential()
client_akv = SecretClient(vault_url=KVUri, credential=credential)
cosmosdb_endpoint = client_akv.get_secret(name="cosmosdb-url").value
conn_str = client_akv.get_secret(name="cosmosdb-connstr").value
# conn_str = os.environ["COSMOS_CONN_STRING"]
azure_openai_endpoint=client_akv.get_secret(name="aoai-endpoint").value
azure_openai_api_key=client_akv.get_secret(name="aoai-api-key").value
azure_openai_api_version = "2024-02-15-preview"

#client = CosmosClient.from_connection_string(conn_str)
cosmos_client = CosmosClient(cosmosdb_endpoint, credential=credential)

# Embeddings for semantic search in long-term memory
emb = init_embeddings("azure_openai:text-embedding-3-small", 
                      api_key=azure_openai_api_key, 
                      azure_openai_api_version=azure_openai_api_version, 
                      azure_endpoint=azure_openai_endpoint)

index_cfg: IndexConfig = {
    "dims": 1536,
    "embed": emb,
    "fields": ["summary", "text"],  # fields inside your stored value dict
}

store = CosmosDBStore(
    client=cosmos_client,
    database_id="langgraph-test-db",
    container_id="memory-test",
    partition_key_path="/namespaceKey",
    index=index_cfg,  # smoke test: no vector index yet
)

# One-time setup (DB + container + indexes/TTL)
store.setup(ttl_seconds=60 * 60 * 24 * 7)  # 7 days, or None for no default TTL

from typing import TypedDict
from datetime import datetime


# Model configuration
# instantiate the chatbot model
model = AzureChatOpenAI(
    model="gpt-4o",
    api_key=azure_openai_api_key,
    api_version="2024-12-01-preview",
    azure_endpoint=azure_openai_endpoint,
    temperature=0.5
)

class ConversationState(MessagesState):
    summary: str | None

builder = StateGraph(ConversationState)

from langchain_core.messages import HumanMessage, AIMessage

def call_model(state: ConversationState, config) -> ConversationState:
    response = model.invoke(state["messages"], config=config)
    # response is an AIMessage
    updated = list(state["messages"]) + [response]
    return {"messages": updated, "summary": state.get("summary")}

def write_memory(state: ConversationState, config) -> ConversationState:
    user_id = config["configurable"]["user_id"].replace(".", "_").replace("@", "_")
    namespace = ("user", user_id, "summaries")

    user_msg = next((m for m in state["messages"] if isinstance(m, HumanMessage)), None)
    assistant_msg = next((m for m in state["messages"] if isinstance(m, AIMessage)), None)

    user_text = user_msg.content if user_msg else ""
    assistant_text = assistant_msg.content if assistant_msg else ""

    store.put(
        namespace,
        key=str(uuid.uuid4()),
        value={
            "summary": f"user: {user_text[:40]}...",
            "text": assistant_text,
            "turn": {
                "messages": [m.model_dump() for m in state["messages"]],
            },
        },
        index=True,
    )
    return state

builder.add_node("llm", call_model)
builder.add_node("write_memory", write_memory)
builder.add_edge(START, "llm")
builder.add_edge("llm", "write_memory")
builder.add_edge("write_memory", END)

graph = builder.compile(store=store)

In [6]:
user_id = "charlesc@partnergem.com"
config = {"configurable": {"thread_id": "1", "user_id": user_id}}
payload = {"messages": [HumanMessage(content="When was France created?")]}
result = graph.invoke(payload, config=config)
print(result["messages"][-1].content)  # assistant answer

France, as a distinct entity, has a long history that evolved over centuries. The origins of modern France can be traced back to the division of the Carolingian Empire. The Treaty of Verdun in 843 AD is often cited as a significant event, as it divided the empire among Charlemagne's grandsons. This treaty led to the formation of West Francia, which roughly corresponds to the territory of modern France.

Over time, West Francia evolved through various dynastic changes, territorial expansions, and political developments to become the Kingdom of France. The Capetian dynasty, beginning with Hugh Capet's coronation in 987 AD, played a crucial role in consolidating power and laying the foundation for the modern French state.

France continued to develop through the Middle Ages, the Renaissance, and into the modern era, with significant events such as the French Revolution in 1789, which transformed it from a monarchy into a republic. Each of these periods contributed to the formation of the 

In [9]:
import json

def fetch_user_memories(user_id: str, *, limit: int = 10):
    # mirror the namespace formatting used when writing
    sanitized = user_id.replace(".", "_").replace("@", "_")
    namespace_prefix = "|".join(("user", sanitized, "summaries"))

    container = store._container  # Cosmos SDK ContainerProxy already created by CosmosDBStore
    query = """
        SELECT c.id,
               c.key,
               c["value"] AS docValue,
               c.created_at,
               c.updated_at
        FROM c
        WHERE STARTSWITH(c.namespaceKey, @ns_prefix)
        ORDER BY c.updated_at DESC
    """
    params = [{"name": "@ns_prefix", "value": namespace_prefix}]
    docs = list(
        container.query_items(
            query=query,
            parameters=params,
            enable_cross_partition_query=True,
            max_item_count=limit,
        )
    )

    for doc in docs:
        print(f"id: {doc['id']}")
        print(f"key: {doc['key']}")
        print(f"created_at: {doc['created_at']}")
        print(json.dumps(doc["docValue"], indent=2))
        print("-" * 60)

    return docs  # full records if you need them programmatically

In [10]:
fetch_user_memories("charlesc@partnergem.com", limit=5)

id: user|charlesc_partnergem_com|summaries::6642524c-fff2-4788-ba63-eee58c2e9bb5
key: 6642524c-fff2-4788-ba63-eee58c2e9bb5
created_at: 2025-11-25T00:22:51.706468+00:00
{
  "summary": "user: When was France created?...",
  "text": "France, as a distinct entity, has a long history that evolved over centuries. The origins of modern France can be traced back to the division of the Carolingian Empire. The Treaty of Verdun in 843 AD is often cited as a significant event, as it divided the empire among Charlemagne's grandsons. This treaty led to the formation of West Francia, which roughly corresponds to the territory of modern France.\n\nOver time, West Francia evolved through various dynastic changes, territorial expansions, and political developments to become the Kingdom of France. The Capetian dynasty, beginning with Hugh Capet's coronation in 987 AD, played a crucial role in consolidating power and laying the foundation for the modern French state.\n\nFrance continued to develop through

[{'id': 'user|charlesc_partnergem_com|summaries::6642524c-fff2-4788-ba63-eee58c2e9bb5',
  'key': '6642524c-fff2-4788-ba63-eee58c2e9bb5',
  'docValue': {'summary': 'user: When was France created?...',
   'text': "France, as a distinct entity, has a long history that evolved over centuries. The origins of modern France can be traced back to the division of the Carolingian Empire. The Treaty of Verdun in 843 AD is often cited as a significant event, as it divided the empire among Charlemagne's grandsons. This treaty led to the formation of West Francia, which roughly corresponds to the territory of modern France.\n\nOver time, West Francia evolved through various dynastic changes, territorial expansions, and political developments to become the Kingdom of France. The Capetian dynasty, beginning with Hugh Capet's coronation in 987 AD, played a crucial role in consolidating power and laying the foundation for the modern French state.\n\nFrance continued to develop through the Middle Ages, t